# Package Pytorch Vision Model using SageMaker EdgeManager and Neo

## Download ResNet18 from TorchVision
Download ResNet18 model from TorchVision and create a model artifact `model.tar.gz`.

In [2]:
import sagemaker

In [3]:
import sys

!{sys.executable} -m pip install --upgrade sagemaker

  Using cached sagemaker-2.94.0.tar.gz (527 kB)
  Preparing metadata (setup.py) ... done
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Created wheel for sagemaker: filename=sagemaker-2.94.0-py2.py3-none-any.whl size=740787 sha256=2ecc384b371bd3917923736da274d99999123a79b9abf741c4f3bfa85f287ce9
  Stored in directory: /root/.cache/pip/wheels/60/9c/3d/36b86becda591c23df6e34d43889d587a0b3aafd1fe76de959
Successfully built sagemaker
  Attempting uninstall: attrs
    Found existing installation: attrs 21.2.0
    Uninstalling attrs-21.2.0:
      Successfully uninstalled attrs-21.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.72.0
    Uninstalling sagemaker-2.72.0:
      Successfully uninstalled sagemaker-2.72.0


Specify the input data shape. For more information, see [Prepare Model for Compilation].(https://docs.aws.amazon.com/sagemaker/latest/dg/neo-compilation-preparing-model.html)

In [4]:
import torch
torch.__version__

'1.8.1'

In [5]:
import torchvision
torchvision.__version__

'0.9.1'

In [6]:
import sagemaker
import torch
import torchvision.models as models
import tarfile

image_classification_model = models.resnet18(pretrained=True)
input_tensor = torch.zeros([1, 3, 224, 224]) 
trace = torch.jit.trace(image_classification_model.float().eval(), input_tensor.float())
trace.save("model.pth")

with tarfile.open("model.tar.gz", "w:gz") as f:
    f.add("model.pth")

[2022-06-08 18:08:13.420 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:33 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-06-08 18:08:13.633 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:33 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


### Setup 

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3
from sagemaker.utils import name_from_base


sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

compilation_job_name = name_from_base("image-classification-neo")
key_prefix = 'horse-or-human'

role = get_execution_role() 
client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

### Upload the model to S3

In [8]:
model_uri = sagemaker_session.upload_data(path="model.tar.gz", key_prefix=key_prefix)
print("S3 Path for Model: ", model_uri)

S3 Path for Model:  s3://sagemaker-us-east-1-670488263423/horse-or-human/model.tar.gz


## Compile Model

In [25]:
compilation_job_name = name_from_base("image-classification-neo")
prefix = key_prefix+'/'+compilation_job_name + "/model"

data_shape = '{"input0":[1,3,224,224]}'
target_device = "ml_c5"
framework = "PYTORCH"
framework_version = "1.8"
compiled_model_path = "s3://{}/{}/output".format(bucket, compilation_job_name)
print("S3 path for compiled model: ", compiled_model_path)

S3 path for compiled model:  s3://sagemaker-us-east-1-670488263423/image-classification-neo-2022-06-08-18-16-10-946/output


In [16]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

sagemaker_model = PyTorchModel(
    model_data=model_uri,
    predictor_cls=Predictor,
    framework_version=framework_version,
    role=role,
    sagemaker_session=sagemaker_session,
    entry_point="inference.py",
    source_dir="code",
    py_version="py3",
    env={"MMS_DEFAULT_RESPONSE_TIMEOUT": "500"},
)

In [27]:
sagemaker_client = boto3.client("sagemaker", region_name=region)
target_arch = "X86_64" # raspberry pi architecture: https://docs.aws.amazon.com/sagemaker/latest/dg/neo-supported-devices-edge-devices.html
target_os = 'LINUX'
response = sagemaker_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        "S3Uri": sagemaker_model.model_data,
        "DataInputConfig": data_shape,
        "Framework": framework,
    },
    OutputConfig={
        "S3OutputLocation": compiled_model_path,
#         "TargetDevice": 'jetson_nano',
        "TargetPlatform": {
            "Arch": target_arch, 
            "Os": target_os
        },
    },
    StoppingCondition={"MaxRuntimeInSeconds": 900},
)
print(response)

In [31]:
print(response)

{'CompilationJobArn': 'arn:aws:sagemaker:us-east-1:670488263423:compilation-job/image-classification-neo-2022-06-08-18-16-10-946', 'ResponseMetadata': {'RequestId': '48766ba9-89ca-4161-9c7a-3fd89153cb5e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '48766ba9-89ca-4161-9c7a-3fd89153cb5e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Wed, 08 Jun 2022 18:19:58 GMT'}, 'RetryAttempts': 0}}


In [32]:
# Poll every 30 sec
import time
while True:
    response = sagemaker_client.describe_compilation_job(
        CompilationJobName=compilation_job_name
    )
    if response["CompilationJobStatus"] == "COMPLETED":
        break
    elif response["CompilationJobStatus"] == "FAILED":
        print(response)
        raise RuntimeError("Compilation failed")
    print("Compiling ...")
    time.sleep(30)
print("Done!")

Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Done!


In [28]:
# compiled_model = sagemaker_model.compile(
#     target_instance_family=None,
#     input_shape=data_shape,
#     job_name=compilation_job_name,
#     role=role,
#     framework=framework.lower(),
#     framework_version=framework_version,
#     output_path=compiled_model_path,
#     target_platform_os='LINUX',
#     target_platform_arch='X86_64',
# )

In [19]:
predictor = compiled_model.deploy(initial_instance_count=1, 
                       instance_type="ml.c5.2xlarge")

INFO:sagemaker:Creating model with name: sagemaker-inference-pytorch-ml-c5-2022-05-18-23-37-17-861
INFO:sagemaker:Creating endpoint-config with name sagemaker-inference-pytorch-ml-c5-2022-05-18-23-37-18-360
INFO:sagemaker:Creating endpoint with name sagemaker-inference-pytorch-ml-c5-2022-05-18-23-37-18-360


--------!

In [21]:
import json
import requests
from IPython.display import Image 
import json
import boto3
import numpy as np

runtime= boto3.client('runtime.sagemaker')
client = boto3.client('sagemaker')

endpoint_desc = client.describe_endpoint(EndpointName=ENDPOINT_NAME)
print(endpoint_desc)
print('---'*60)

{'EndpointName': 'sagemaker-inference-pytorch-ml-c5-2022-05-18-23-37-18-360', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:670488263423:endpoint/sagemaker-inference-pytorch-ml-c5-2022-05-18-23-37-18-360', 'EndpointConfigName': 'sagemaker-inference-pytorch-ml-c5-2022-05-18-23-37-18-360', 'ProductionVariants': [{'VariantName': 'AllTraffic', 'DeployedImages': [{'SpecifiedImage': '785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-inference-pytorch:1.8-cpu-py3', 'ResolvedImage': '785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-inference-pytorch@sha256:8e67a1672fb5dfc0a527362bcc6c98f37d9cd8d89a30872b45824c501d3c3fc0', 'ResolutionTime': datetime.datetime(2022, 5, 18, 23, 37, 19, 533000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2022, 5, 18, 23, 37, 18, 888000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2022, 5, 18, 23, 41, 3, 

In [22]:
import numpy as np
import json

with open("horse_cart.jpg", "rb") as f:
    payload = f.read()
    payload = bytearray(payload)

response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME, 
                                   ContentType='application/octet-stream', 
                                   Body=payload, 
                                   Accept = 'application/json')
result = response['Body'].read()
result = json.loads(result)
print(result)

{'label': "'horse cart, horse-cart'", 'probability': '98.93%'}


In [25]:
# delete endpoint after testing the inference
import boto3

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=ENDPOINT_NAME)

{'ResponseMetadata': {'RequestId': '78a6ab39-0c59-4f0e-997d-b8f87aebce54',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '78a6ab39-0c59-4f0e-997d-b8f87aebce54',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 04 Jun 2022 18:40:45 GMT'},
  'RetryAttempts': 0}}

## Package Model using Sagemaker Edge Manager

In this section, we will walk through packaging of image classification resnet model. 

### Package Resnet Model

Before we can deploy the compiled model to edge devices, we need to package the model using SageMaker Edge Manager cloud service.


In [33]:
packaged_model_name = "resnet"
model_version = "1.0"
compilation_job_name

'image-classification-neo-2022-06-08-18-16-10-946'

In [34]:
import os
s3_edge_output_location = os.path.join(compiled_model_path,'edgemanager')
s3_edge_output_location

's3://sagemaker-us-east-1-670488263423/image-classification-neo-2022-06-08-18-16-10-946/output/edgemanager'

In [35]:
import time

packaging_job_name = compilation_job_name + "-packaging-ggv2"
# The name you want your Greengrass component to have.
component_name = "SagemakerEdgeManager" + packaging_job_name

# add Greengrass v2 component
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        "S3OutputLocation": s3_edge_output_location,
        # "PresetDeploymentType":"GreengrassV2Component",
        # "PresetDeploymentConfig":"{\"ComponentName\":\"sagemaker-em-iot-component\", \"ComponentVersion\":\"1.0.2\"}"
    },
    ModelName=packaged_model_name,
    ModelVersion=model_version,
    EdgePackagingJobName=packaging_job_name,
    CompilationJobName=compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(
        EdgePackagingJobName=packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        print(job_status)
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)
print("Done!")

{'ResponseMetadata': {'RequestId': '767cd0ad-eb0e-430f-be00-f1b99387fb9b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '767cd0ad-eb0e-430f-be00-f1b99387fb9b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Wed, 08 Jun 2022 18:27:18 GMT'}, 'RetryAttempts': 0}}
Packaging ...
Done!


### Create a Device Fleet for EdgeManager

**Note** : Select the IoT Role ARN created at the begining of the chapter, before running the next step.

In [ ]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
account_id

In [43]:
# Create a Device Fleet for EdgeManager
s3_device_fleet_output = os.path.join(s3_edge_output_location, 'fleet')
iot_role_arn = f'arn:aws:iam::{account_id}:role/SageMakerGreenGrassV2MinimalResourceRole'
device_fleet_name = "mydevicefleet"

In [ ]:
sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=iot_role_arn, # IoT Role ARN specified in previous step
    OutputConfig={
        'S3OutputLocation': s3_device_fleet_output
    }
)

### Register your device

**Important Note:** Register your device with the same name as your AWS IoT thing name created during the AWS IoT Greengrass V2 setup.

In [44]:
# Device name should be 36 characters
device_name = 'mything'

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {
            "DeviceName": device_name,
            "IotThingName": device_name
        }
    ]
)

{'ResponseMetadata': {'RequestId': '48946074-4694-4645-8ebb-e40953b20675',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '48946074-4694-4645-8ebb-e40953b20675',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 04 Jun 2022 19:57:04 GMT'},
  'RetryAttempts': 0}}